In [ ]:
import os.path

import numpy as np
import numpy.linalg as la
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4

import bin_model as bm

# Absolute, Relative, and Mixed Errors

In [ ]:
KERNEL_FILE_NAME = "kernels/Hall_ScottChen_kernel_nb336.nc"
LOGT_FILE_NAME_TEMPLATE = \
    os.path.join("sensitivity_experiments",
                 "experiment_d{}nu07_c0-3_r0-3_logt.nc")
IDT_FILE_NAME_TEMPLATE = \
    os.path.join("sensitivity_experiments",
                 "experiment_d{}nu07_c0-3_r0-3_idt.nc")
QTLT_FILE_NAME_TEMPLATE = \
    os.path.join("sensitivity_experiments",
                 "experiment_d{}nu07_c0-3_r0-3_qtlt.nc")

In [ ]:
with nc4.Dataset(KERNEL_FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    _, kernel, _, ktens = netcdf_file.read_cgk()

In [ ]:
first_file_name = LOGT_FILE_NAME_TEMPLATE.format(15)
with nc4.Dataset(first_file_name, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp = netcdf_file.read_full_experiment([ktens])
nt = exp.num_time_steps

In [ ]:
wv_m6 = exp.mass_grid.moment_weight_vector(6)

In [ ]:
const = exp.constants
m6_scale = const.mass_conc_scale * const.std_diameter**6 / const.std_mass

In [ ]:
DS = np.array([15 + 3*i for i in range(6)])

In [ ]:
nds = len(DS)

In [ ]:
m6s_logt = np.zeros((nds, nt))
m6_covs_logt = np.zeros((nds, nt))
for i in range(nds):
    exp_file_name = LOGT_FILE_NAME_TEMPLATE.format(DS[i])
    if not os.path.isfile(exp_file_name):
        m6s_logt[i,:] = np.nan
        m6_covs_logt[i,:] = np.nan
        continue
    with nc4.Dataset(exp_file_name, "r") as nc:
        netcdf_file = bm.NetcdfFile(nc)
        exp = netcdf_file.read_full_experiment([ktens])
    m6s_logt[i,:], m6_covs_logt[i,:] = exp.get_moments_and_covariances(wv_m6)

In [ ]:
m6_logt_worsts = np.sqrt(m6_covs_logt).max(axis=1) * m6_scale

In [ ]:
m6s_idt = np.zeros((nds, nt))
m6_covs_idt = np.zeros((nds, nt))
for i in range(nds):
    exp_file_name = IDT_FILE_NAME_TEMPLATE.format(DS[i])
    if not os.path.isfile(exp_file_name):
        m6s_idt[i,:] = np.nan
        m6_covs_idt[i,:] = np.nan
        continue
    with nc4.Dataset(exp_file_name, "r") as nc:
        netcdf_file = bm.NetcdfFile(nc)
        exp = netcdf_file.read_full_experiment([ktens])
    m6s_idt[i,:], m6_covs_idt[i,:] = exp.get_moments_and_covariances(wv_m6)

In [ ]:
m6_idt_worsts = np.sqrt(m6_covs_idt).max(axis=1) * m6_scale

In [ ]:
m6s_qtlt = np.zeros((nds, nt))
m6_covs_qtlt = np.zeros((nds, nt))
for i in range(nds):
    exp_file_name = QTLT_FILE_NAME_TEMPLATE.format(DS[i])
    if not os.path.isfile(exp_file_name):
        m6s_qtlt[i,:] = np.nan
        m6_covs_qtlt[i,:] = np.nan
        continue
    with nc4.Dataset(exp_file_name, "r") as nc:
        netcdf_file = bm.NetcdfFile(nc)
        exp = netcdf_file.read_full_experiment([ktens])
    m6s_qtlt[i,:], m6_covs_qtlt[i,:] = exp.get_moments_and_covariances(wv_m6)

In [ ]:
m6_qtlt_worsts = np.sqrt(m6_covs_qtlt).max(axis=1) * m6_scale

In [ ]:
plt.semilogy(DS, m6_idt_worsts)
plt.semilogy(DS, m6_logt_worsts)
plt.semilogy(DS, m6_qtlt_worsts)

In [ ]:
plt.plot(exp.times, np.sqrt(m6_covs_idt[1,:]) * m6_scale)
plt.plot(exp.times, np.sqrt(m6_covs_logt[1,:]) * m6_scale)
plt.plot(exp.times, np.sqrt(m6_covs_qtlt[1,:]) * m6_scale)

In [ ]:
plt.plot(exp.times, m6_scale * m6s_qtlt[1,:])